In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/pure_datatape_450x8x8_train.h5"
valid_tape = "Z:/rppg/pure_datatape_450x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()

In [3]:
m1 = M_1()
m1.compile(optimizer='Nadam', loss='mae')
m1.build(input_shape=(None, 450, 8, 8, 3))
print(f'Flops per frame:{get_flops(m1, input_sig=[tf.TensorSpec([1, 450, 8, 8, 3])])/450:.0f}')
m1.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:260557
Model: "m_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           12352     
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  10272     
_________________

In [4]:
m1.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m1_pure.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m1.load_weights('../weights/m1_pure.h5')

Epoch 1/10
81/81 [==============================] - 8s 36ms/step - loss: 0.9049 - val_loss: 0.9653

Epoch 00001: val_loss improved from inf to 0.96532, saving model to ../weights\m1_pure.h5
Epoch 2/10
81/81 [==============================] - 1s 18ms/step - loss: 0.7116 - val_loss: 0.7577

Epoch 00002: val_loss improved from 0.96532 to 0.75770, saving model to ../weights\m1_pure.h5
Epoch 3/10
81/81 [==============================] - 1s 18ms/step - loss: 0.5173 - val_loss: 0.6883

Epoch 00003: val_loss improved from 0.75770 to 0.68834, saving model to ../weights\m1_pure.h5
Epoch 4/10
81/81 [==============================] - 2s 19ms/step - loss: 0.4341 - val_loss: 0.6109

Epoch 00004: val_loss improved from 0.68834 to 0.61093, saving model to ../weights\m1_pure.h5
Epoch 5/10
81/81 [==============================] - 2s 19ms/step - loss: 0.3924 - val_loss: 0.6411

Epoch 00005: val_loss did not improve from 0.61093
Epoch 6/10
81/81 [==============================] - 2s 19ms/step - loss: 0.35

In [ ]:
m1.load_weights('../weights/m1_pure.h5')

In [7]:
eval_on_dataset(test_set_CCNU, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_PURE_CCNU.h5')
get_metrics('../results/ours_PURE_CCNU.h5', use_filter=False)

100%|██████████| 179/179 [00:25<00:00,  6.91it/s]


{'Sliding window': {'MAE': 5.823, 'RMSE': 15.063, 'R': 0.40767},
 'Whole video': {'MAE': 2.893, 'RMSE': 10.172, 'R': 0.60115}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_PURE_CCNU_rPPG.h5')
get_metrics('../results/ours_PURE_CCNU_rPPG.h5', use_filter=False)

100%|██████████| 54/54 [00:05<00:00,  9.49it/s]


{'Sliding window': {'MAE': 6.487, 'RMSE': 16.197, 'R': 0.37994},
 'Whole video': {'MAE': 5.049, 'RMSE': 14.564, 'R': 0.39071}}

In [8]:
eval_on_dataset(test_set_UBFC_rPPG2, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_PURE_UBFC.h5')
get_metrics('../results/ours_PURE_UBFC.h5',use_filter=False)

100%|██████████| 42/42 [00:02<00:00, 19.93it/s]


{'Sliding window': {'MAE': 2.357, 'RMSE': 7.418, 'R': 0.92078},
 'Whole video': {'MAE': 1.158, 'RMSE': 1.857, 'R': 0.99448}}